### Import pandas library

In [37]:
import pandas as pd

### Read recipe json (populated from ChatGPT)

In [38]:
recipes = pd.read_json('../seeds/recipes.json', orient='records')
recipes = pd.json_normalize(recipes['recipes'], max_level=0)[['name', 'author', 'meal', 'dish', 'cuisine', 'servings', 'time_required', 'ingredients', 'directions']]
recipes = recipes.reset_index().rename(columns={'index': 'recipe_id'})

### Retrieve users from the recipe list to populate User table

In [39]:
users = recipes.drop_duplicates('author')['author'].reset_index(drop=True)
users = users.reset_index().rename(columns={'index': 'user_id', 'author': 'name'})

### Retrieve ingredients from the recipe list to populate Ingredient table

In [40]:
ingredients = recipes.explode('ingredients')
ingredients = ingredients.reset_index()
ingredients = ingredients.drop(columns='name')
split = pd.DataFrame(ingredients['ingredients'].to_list(), columns = ['name', 'amount', 'metric'])
ingredients = pd.concat([ingredients, split], axis=1)[['recipe_id', 'name', 'amount', 'metric']]
ingredients = ingredients.reset_index().rename(columns={'index': 'ingredient_id'})

### Use IDs from Ingredient and User table to add references in Recipe table

In [41]:
recipes['ingredients'] = recipes['recipe_id'].apply(lambda x: ingredients.loc[ingredients['recipe_id'] == x, 'ingredient_id'].to_list())

In [42]:
recipes['user_id'] = recipes['author'].apply(lambda x: users.loc[users['name']== x, 'user_id'].iloc[0])
recipes = recipes.drop(columns = 'author')

In [43]:
# recipes.name.drop_duplicates().count()
recipes.name.count()

50

### Verify tables have been set up properly

In [46]:
# recipes.head()

In [47]:
# ingredients['name'].unique()

### QA/QC to make sure the numbers are isolated

In [48]:
ingredients['amount'].unique()

array(['2', '1', '3', '', '4', '6', '1/4', '1/2', '9', '28', '15', '14',
       '250', '200', '500', '8', '1 1/2', '400', '300', '100', '800',
       '20', '12', '5', '150'], dtype=object)

### Write to tables to individual CSVs 

In [49]:
recipes.to_csv('recipes.csv')

In [50]:
ingredients.to_csv('ingredients.csv')

In [51]:
users.to_csv('users.csv')